In [1]:
import tensorflow as tf

C:\Users\wjssm\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
sess=tf.Session()
a=tf.constant(10)
print(sess.run(a))

10


In [5]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./MNIST_data", one_hot=True)

Extracting ./MNIST_data\train-images-idx3-ubyte.gz
Extracting ./MNIST_data\train-labels-idx1-ubyte.gz
Extracting ./MNIST_data\t10k-images-idx3-ubyte.gz
Extracting ./MNIST_data\t10k-labels-idx1-ubyte.gz


In [6]:
print(mnist.train.images[0]) #mnist train data의 image중 첫번째

[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         

In [7]:
print(mnist.train.labels[15]) #labels에는 0~9까지 어떤 숫자인지 저장. 이 숫자는 7이다

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


### Varaibles

In [8]:
x=tf.placeholder(tf.float32, [None,784]) #784개의 메모리 잡아줌. placeholder는 이후넣어줄 MNIST값

W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10])) #bias
y_ = tf.placeholder(tf.float32, [None, 10])

### Softmax

In [9]:
y=tf.nn.softmax(tf.matmul(x,W)+b) #matmul : 행렬곱, nn.softmax : softmax관련

### Model

In [7]:
#cross_entropy 를 최소화하는 방향으로.cost가 분포로 나타난다고
#하나의 배치에 대해서 cross_entropy를 구해서 weight를 줄이는 방향으로

In [8]:
#GradientDescentOptimizer(0.5)<-running rate조절

In [10]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [11]:
init = tf.global_variables_initializer()

In [12]:
sess=tf.Session() #모든 작업을 session을 통해 run
sess.run(init)

In [13]:
for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100) #batch 100개씩 1000번
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

In [14]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1)) #armax(y,1) : y에서 가장 큰 것은 1로 바꿔라. euql : 같으면True, 틀리면 False


In [15]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)) #숫자로 바꿔서 평균내
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9196


# CNN

In [21]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1) #정규분포로
    return tf.Variable(initial)


In [22]:
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape) #cnn은 0으로 초기화하지 말 것
    return tf.Variable(initial)

In [23]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME') #padding same : 원래 shape모양유지되도록

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [24]:
sess = tf.InteractiveSession() #session 여러개

In [25]:
W_conv1 = weight_variable([5, 5, 1, 32]) #5X5 필터 32개
b_conv1 = bias_variable([32])

x_image = tf.reshape(x, [-1,28,28,1])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1) #reslu로 activation
h_pool1 = max_pool_2x2(h_conv1) #conv layer를 pooling처리 , h_pool1 : 14X14X32

In [26]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [27]:
W_fc1 = weight_variable([7 * 7 * 64, 1024]) #1024 는 내 맘대로 2의 제곱형태로
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1) #1024개의 뉴런을 가지고 있는 featuremap

In [28]:
keep_prob = tf.placeholder(tf.float32) #keep_prob 정해진 비율빼고 나머지는 지워버림. 그대로 shape 는 1024개임
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob) 

In [29]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

In [30]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv),
reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy) #AdamOptimizer가 gradient보다 더좋음

In [31]:
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [32]:
sess.run(tf.global_variables_initializer())
for i in range(5000):
    batch = mnist.train.next_batch(50)
    if i%100 == 0:
        train_accuracy = accuracy.eval( feed_dict={x:batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
    train_step.run(session = sess, feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5}) #keep_porb : 다양한

step 0, training accuracy 0.04


KeyboardInterrupt: 

In [ ]:
print("test accuracy %g"%accuracy.eval( feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))